# Калибровка

## Импорт необходимых модулей/функций

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from pixelpoint.calibration import load_images, load_marker_coords, save_calibration_params, calculate_CM, calculate_RT

## Константы

_cam1_ - левая камера  
_cam2_ - правая камера

In [ ]:
pattern_params = {
    's_markers_dist': 33e-3 / 6.0,  # real distance between markers in small calibration plane (in m)
    'm_markers_dist': 67e-3 / 6.0,  # real distance between markers in medium calibration plane (in m)
    'grid_shape': (7, 7)
}
camera_params = {
    'focal_length_m': 80e-3,
    'img_resolution': (5120, 4096),
    'pixel_sz_x': 4.5e-6,  # in m
    'pixel_sz_y': 4.5e-6  # in m
}
stereo_params = {
    'baseline': 412e-3,  # in m
    'center_dist_to_obj': 639e-3,  # in m
    'cam_dist_to_obj': 650e-3,  # in m
}

## Подготовка данных

Вручную отметил координаты маркерных точек на досках, т.к. иначе получить координаты всех точек на маркерной доске, чтобы они формировали матрицу не удавалось (пробовал это сделать с помощью выделения Blob-ов, binary threshold, Hough Circle Transform, использовать модели на нейронных сетях - U-Net). Выделить маркеры в целом получалось, однако затем всё равно требуется вручную сопоставить их с координатами в системе координат относительно маркерной доски.

Итак, координаты маркерных точек находятся в файле *markers_coords.json*. На каждом изображении 49 точек, формирующих матрицу 7 на 7. Точки затем сопоставляются с координатами самой доски в реальном 3D пространстве *real_markers_coords*.

In [ ]:
imgs_folder = Path('calibration_images_markers')
cam1_imgs, cam2_imgs = load_images(imgs_folder)

In [ ]:
plt.figure(figsize=(20, 10)) 
for i, img in enumerate(cam1_imgs.values()):
    plt.subplot(1, len(cam1_imgs), i + 1) 
    plt.imshow(cv.cvtColor(img, cv.COLOR_GRAY2RGB))
    plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(20, 10)) 
for i, img in enumerate(cam2_imgs.values()):
    plt.subplot(1, len(cam2_imgs), i + 1) 
    plt.imshow(cv.cvtColor(img, cv.COLOR_GRAY2RGB))
    plt.axis('off')
plt.show()

Создадим словарь, в котором будет определено расстояние между маркерами для каждого изображения

In [ ]:
img_name_to_markers_dist = {}

for img_name in cam1_imgs.keys():
    if img_name == 'cam2_4':
        img_name_to_markers_dist[img_name] = pattern_params['s_markers_dist']
    else:
        img_name_to_markers_dist[img_name] = pattern_params['m_markers_dist']

for img_name in cam2_imgs.keys():
    if img_name == 'cam1_4':
        img_name_to_markers_dist[img_name] = pattern_params['s_markers_dist']
    else:
        img_name_to_markers_dist[img_name] = pattern_params['m_markers_dist']

Загрузка координат маркерных точек для калибровочных изображений

In [ ]:
markers_file_path = Path('calibration_params/markers_coords.json')
markers_coords = load_marker_coords(markers_file_path)

In [ ]:
def plot_img_w_markers(img, markers):
    plt.figure(figsize=(10, 10))
    plt.imshow(cv.cvtColor(img, cv.COLOR_GRAY2RGB))
    for marker_coords in np.array(markers, dtype=np.float32):
        plt.scatter(marker_coords[0], marker_coords[1], color='red', s=10)
    plt.show()

### Пример размеченного изображения

In [ ]:
img_name = 'cam2_4'
img = cam1_imgs[img_name]
plot_img_w_markers(img, markers_coords[img_name])

## Поиск внутренних параметров камер

### Вычисление camera matrix

Camera matrix можно вычислить исходя из следующих параметров:
1. фокусное расстояние: 80mm
2. размер пикселя: 4.5 µm × 4.5 µm
3. разрешение изображения: 5120 × 4096 

In [ ]:
CM = calculate_CM(**camera_params)
print(f'Camera matrix:\n{CM}')

### Поиск вектора искажений

У нас уже есть матрица камеры, вычислим коэффициенты искажения с использованием калибровочных изображений и координат маркерных точек на них (*calibration_images_markers/* и *markers_coords.json*).

calibrateCamera принимает начальное предположение для матрицы камеры (CM_initial_guess) и вычисляет только вектор искажения при использовании определенных флагов.

In [ ]:
def calculate_distortion(imgs, img_resolution, 
                         markers_coords, markers_dist, grid_shape,
                         CM, dist_initial_guess=None):
    rows, columns = grid_shape

    real_markers = []  # 3D markers coords in real world for each image
    img_markers = []  # 2D markers coords in image plane for each image
    
    for img_name in imgs:
        # Get the 2D image marker coordinates
        markers = np.array(markers_coords[img_name], dtype=np.float32)
        img_markers.append(markers)

        # Create 3D real-world coordinates grid based on the specific markers_dist for this image
        real_markers_grid_coords = np.zeros((rows * columns, 3), np.float32)
        real_markers_grid_coords[:, :2] = np.mgrid[0:rows, 0:columns].T.reshape(-1, 2)
        real_markers_grid_coords *= markers_dist[img_name]
    
        real_markers.append(real_markers_grid_coords)

    # Calibration flags: use initial guesses, fix focal length and principal point
    flags = cv.CALIB_USE_INTRINSIC_GUESS + cv.CALIB_FIX_FOCAL_LENGTH + cv.CALIB_FIX_PRINCIPAL_POINT
    ret, _, dist, _, _ = cv.calibrateCamera(real_markers, img_markers, img_resolution, 
                                            CM, dist_initial_guess, 
                                            flags=flags)

    print(f'RMSE: {ret}')
    return dist

In [ ]:
dist = calculate_distortion(
    {**cam1_imgs, **cam2_imgs}, camera_params['img_resolution'], 
    markers_coords, img_name_to_markers_dist, pattern_params['grid_shape'], 
    CM, np.array([0., 0., 0., 0., 0.])
)
print(f'Distortion vector:\n{dist}')

### Поиск всех внутренних параметров с помощью OpenCV calibrateCamera

При ситуации, когда мы не знаем никаких параметров камеры: фокусное расстояние, размеры пикселя. Можно получить оценку внутренних параметров с помощью *cv2.calibrateCamera*.

In [ ]:
def calculate_intrinsic(imgs, img_resolution, 
                         markers_coords, markers_dist, grid_shape):
    rows, columns = grid_shape

    real_markers = []  # 3D markers coords in real world for each image
    img_markers = []  # 2D markers coords in image plane for each image
    
    for img_name in imgs:
        # Get the 2D image marker coordinates
        markers = np.array(markers_coords[img_name], dtype=np.float32)
        img_markers.append(markers)

        # Create 3D real-world coordinates grid based on the specific markers_dist for this image
        real_markers_grid_coords = np.zeros((rows * columns, 3), np.float32)
        real_markers_grid_coords[:, :2] = np.mgrid[0:rows, 0:columns].T.reshape(-1, 2)
        real_markers_grid_coords *= markers_dist[img_name]
    
        real_markers.append(real_markers_grid_coords)

    ret, CM, dist, _, _ = cv.calibrateCamera(real_markers, img_markers, img_resolution, None, None)

    print(f'RMSE: {ret}')
    return CM, dist

In [ ]:
CM, dist = calculate_intrinsic(
    {**cam1_imgs, **cam2_imgs}, camera_params['img_resolution'], 
    markers_coords, img_name_to_markers_dist, pattern_params['grid_shape'])

print(f'Camera matrix:\n{CM}')
print(f'Distortion vector:\n{dist}')

## Стерео-калибровка

### Manual estimation of extrinsic parameters for a given optical system configuration

Рассчитаем матрицы поворота и смещения двух систем координат камер

In [ ]:
R, T = calculate_RT(stereo_params['baseline'], stereo_params['cam_dist_to_obj'])

print(f'Rotation matrix:\n{R}')
print(f'Translation vector:\n{T}')

### Stereo-calibration using the calibration images

In [ ]:
def stereo_calibrate(cam1_imgs, cam2_imgs, img_resolution, 
                     markers_coords, markers_dist, grid_shape, 
                     CM, dist,
                     R_guess=None, T_guess=None):
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 1000, 1e-6)
    
    rows, columns = grid_shape

    real_markers = []  # 3D points in real world space
    cam1_img_markers = []
    cam2_img_markers = []

    for cam1_img_name, cam2_img_name in zip(cam1_imgs, cam2_imgs):
        cam1_markers = markers_dist[cam1_img_name]

        # 3D world coordinates for each grid point based on markers distance for each image
        real_markers_grid_coords = np.zeros((rows * columns, 3), np.float32)
        real_markers_grid_coords[:, :2] = np.mgrid[0:rows, 0:columns].T.reshape(-1, 2)
        real_markers_grid_coords *= cam1_markers

        real_markers.append(real_markers_grid_coords)

        cam1_img_markers.append(np.array(markers_coords[cam1_img_name], dtype=np.float32))
        cam2_img_markers.append(np.array(markers_coords[cam2_img_name], dtype=np.float32))

    stereocalibration_flags = cv.CALIB_FIX_INTRINSIC + cv.CALIB_USE_EXTRINSIC_GUESS
    
    ret, _, _, _, _, R, T, E, F, _, _, _ = cv.stereoCalibrateExtended(
        real_markers, 
        cam1_img_markers, 
        cam2_img_markers, 
        CM, dist,
        CM, dist,
        img_resolution,
        R_guess, T_guess,
        criteria=criteria, 
        flags=stereocalibration_flags
    )
 
    print(f"Reprojection Error (RMSE): {ret}")
    return R, T, E, F

R: The relative rotation matrix between the first and second cameras. This matrix describes how the second camera is rotated relative to the first camera.

T: The translation vector between the first and second cameras. This vector describes how far and in which direction the second camera is from the first camera.

E: The essential matrix. This matrix encodes the rotation and translation between the two cameras, and it can be used to compute the epipolar geometry (constraints for stereo matching).

F: The fundamental matrix. This is another way of representing the geometric relationship between the two cameras. It defines the epipolar lines in stereo matching.

In [ ]:
R, T, E, F = stereo_calibrate(cam1_imgs, cam2_imgs, camera_params['img_resolution'],
                              markers_coords, img_name_to_markers_dist, pattern_params['grid_shape'],
                              CM, dist, R, T)

print(f'Rotation matrix:\n{R}')
print(f'Translation vector:\n{T}')

Проверим baseline и углы поворота для полученных матриц

In [ ]:
def rotation_matrix_to_euler_angles(R):
    sy = np.sqrt(R[0, 0] ** 2 + R[1, 0] ** 2)
    
    singular = sy < 1e-6
    
    if not singular:
        yaw = np.arctan2(R[2, 1], R[2, 2])
        pitch = np.arctan2(-R[2, 0], sy)
        roll = np.arctan2(R[1, 0], R[0, 0])
    else:
        yaw = np.arctan2(-R[1, 2], R[1, 1])
        pitch = np.arctan2(-R[2, 0], sy)
        roll = 0

    return np.degrees(np.array([yaw, pitch, roll]))

def show_baseline_and_rotation(T_calculated, R_calculated, real_baseline, R_real=None):
    calculated_baseline = np.linalg.norm(T)
    
    print(f"Calculated baseline: {calculated_baseline:.2f} m")
    print(f"Real baseline: {real_baseline:.2f} m")
    
    baseline_difference = np.abs(calculated_baseline - real_baseline)
    print(f"Baseline difference: {baseline_difference:.2f} m")
    
    euler_angles_calculated = rotation_matrix_to_euler_angles(R_calculated)
    print("\nCalculated Euler angles (Yaw, Pitch, Roll):")
    print(f"Yaw: {euler_angles_calculated[0]:.2f} degrees")
    print(f"Pitch: {euler_angles_calculated[1]:.2f} degrees")
    print(f"Roll: {euler_angles_calculated[2]:.2f} degrees")

In [ ]:
show_baseline_and_rotation(T, R, stereo_params['baseline'])

## Export the camera's intrinsic and extrinsic parameters

In [ ]:
save_calibration_params(CM, dist, R, T, E, F, 'calibration_params/calib_params_real.json')